# The Wilson-Cowan model

In this notebook, the basic use of the implementation of the Wilson-Cowan (`wc`) model is presented. 

In the `wc` model, the activity of a particular brain region is defined by a coupled system of excitatory (E) and inhibitory (I) neuronal populations with the mean firing rates of the E and I pools being the dynamic variables, as first described by Wilson and Cowan in 1972 ( H.R. Wilson and J.D. Cowan. *Excitatory and inhibitory interactions in localized populations of model neurons*. Biophys. J., 12:1–24 (1972)) 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import scipy
import neurolib.utils.devutils as du

from neurolib.models.wc import WCModel

import neurolib.utils.loadData as ld
import neurolib.utils.functions as func
from neurolib.utils.loadData import Dataset
from scipy.io import loadmat
from neurolib.utils.loadData import filterSubcortical

# a nice color map
plt.rcParams['image.cmap'] = 'plasma'

In [2]:
data_path = "/Users/pamirozsuna/Desktop/pamir_ma/SCZ-FC-modelling"

In [3]:
def averageMat(Mat):
    avMat = np.zeros((94,94))
    for i in range(len(Mat)):
        avMat = avMat + Mat[i]
    avMat = avMat/len(Mat)
    return avMat

In [4]:
C_data = np.load(data_path+"/AvgCmatrixSCZ.npy")
D_data = np.load(data_path+"/AvgDmatrixSCZ.npy")

In [5]:
Cmat_tmp = averageMat(C_data)
Dmat_tmp = averageMat(D_data)

In [6]:
Cmat = filterSubcortical(Cmat_tmp)
Dmat = filterSubcortical(Dmat_tmp)

In [7]:
FC_data = np.load(data_path+"/AvgFmatrixSCZ.npy")
timeseries = np.load(data_path+"/AvgTmatrixSCZ.npy")

In [8]:
FC_tmp = averageMat(FC_data)
FCs = filterSubcortical(FC_tmp)

In [9]:
#wc.params['exc_ext'] = 0.45
#wc.params['c_excinh'] = 10.1
#wc.params['c_inhexc'] = 9.55
#wc.params['c_inhinh'] = 0
#wc.params['K_gl'] = 1.84
#wc.params['sigma_ou'] = 4.28e-05

#wc.params['exc_ext'] = 0.45
#wc.params['c_excinh'] = 10.1
#wc.params['c_inhexc'] = 9.95
#wc.params['c_inhinh'] = 0
#wc.params['K_gl'] = 1.84
#wc.params['sigma_ou'] = 4.28e-05
total_gbc_model = []
total_scores_model = []
i_list = []

input = np.linspace(0, 1.5, 1000) 

for i in range(len(input)): 
    wc = WCModel(Cmat = Cmat, Dmat = Dmat)
    wc.params['duration'] = 40*1000
    print(i)
    print(input[i])
    wc.params['exc_ext'] = input[i]
    wc.params['c_excinh'] = 10.33
    wc.params['c_inhexc'] = 9.67
    wc.params['c_inhinh'] = 0
    wc.params['K_gl'] = 1.84
    wc.params['sigma_ou'] = 4.28e-05
    wc.run(bold = True)
    gbc_model = np.mean(func.fc(wc.BOLD.BOLD[:, wc.BOLD.t_BOLD>10000]))
    total_gbc_model.append(gbc_model)
    print(gbc_model)
    scores = [func.matrix_correlation(func.fc(wc.BOLD.BOLD[:, wc.BOLD.t_BOLD>10000]), FCs)]
    total_scores_model.append(np.mean(scores))
    i_list.append(i)

0
0.0
0.9992279293901352
1
0.0015015015015015015
0.9992746835530198
2
0.003003003003003003
0.9993270148128685
3
0.0045045045045045045
0.999157828404422
4
0.006006006006006006
0.9991729853392417
5
0.0075075075075075074
0.9992160844935056
6
0.009009009009009009
0.9992392834435967
7
0.01051051051051051
0.9992919677871438
8
0.012012012012012012
0.9993262793868505
9
0.013513513513513514


KeyboardInterrupt: 

In [ ]:
print(total_gbc_model)
print(total_scores_model)

In [ ]:
print(input)

In [ ]:
plt.plot(input, total_gbc_model)
plt.show()